In [124]:
import pandas as pd
import numpy as np
import re

### ** Đọc dữ liệu **

In [125]:
countries = pd.read_csv('./rawdata/country_rankings.csv')
clubs = pd.read_csv('./rawdata/club_tournament.csv')
players = pd.read_csv('./rawdata/players_data.csv')

In [126]:
players = players.merge(countries[['name', 'points']], how='left', left_on='country', right_on='name')
players = players.rename(columns={'points': 'point_of_tournament', 'name_x': 'name'})
players = players.drop('name_y', axis=1)

### ** Missing data **

In [127]:
players.isna().sum()

name                            0
market_value                    0
club                            0
country                         0
league                          0
nationality                     0
birth_date                      0
age                             0
height                          0
preferred_foot                  0
position                        0
shirt_number                    0
attacking                       0
technical                       0
tactical                        0
defending                       0
creativity                      0
saves                           0
anticipation                    0
ball_distribution               0
aerial                          0
avg_score_last_12_months        0
appearances_last_12_months      0
point_of_tournament           607
dtype: int64

In [128]:
players[players['point_of_tournament'].isna()]['country'].unique()

array(['England Amateur', 'Switzerland Amateur', 'Czechia', 'Türkiye',
       'Belgium Amateur'], dtype=object)

In [129]:
players[players['point_of_tournament'].isna()].head(3)

,name,market_value,club,country,league,nationality,birth_date,age,height,preferred_foot,...,tactical,defending,creativity,saves,anticipation,ball_distribution,aerial,avg_score_last_12_months,appearances_last_12_months,point_of_tournament
1193,Cesare Casadei,10.4M €,Chelsea U21,England Amateur,"Premier League 2, Division 1",ITA,10 Jan 2003,21 yrs,186 cm,Right,...,47,51,50,0,0,0,0,7.01,41,NaN
1414,Oliver Arblaster,440K €,Sheffield United U21,England Amateur,Professional Development League,ENG,5 May 2004,19 yrs,180 cm,Right,...,47,48,51,0,0,0,0,6.98,28,NaN
5620,Francis Momoh,835K €,Grasshopper Club Zurich II,Switzerland Amateur,Erste Liga Classic,NGR,25 Mar 2001,23 yrs,187 cm,Right,...,33,26,50,0,0,0,0,6.90,21,NaN


In [130]:
players = players[~players['point_of_tournament'].isna()]

In [131]:
players[players['point_of_tournament'].isna()].head(3)

,name,market_value,club,country,league,nationality,birth_date,age,height,preferred_foot,...,tactical,defending,creativity,saves,anticipation,ball_distribution,aerial,avg_score_last_12_months,appearances_last_12_months,point_of_tournament


In [132]:
players = players.drop('point_of_tournament', axis=1)

### ** Xử lý dữ liệu **

#### 1. Xử lý dữ liệu 'market_value'

In [133]:
players[['name', 'market_value']].head(3)

,name,market_value
0,Luciano Gondou,7.4M €
1,Maximiliano Romero,2.5M €
2,Gastón Verón,1.2M €


In [134]:
all_values = players['market_value']
unique_letters = set()

for value in all_values:
    letters = re.findall('[A-Za-z]', value)
    unique_letters.update(letters)
print(unique_letters)

{'K', 'M'}


In [135]:
# Chuyển đổi cột giá 'market value' theo giá tương ứng
players['market_value'] = players['market_value'].apply(lambda x: x.split(" ")[0].replace("M", "000"))
players['market_value'] = players['market_value'].apply(lambda x: x.split(" ")[0].replace("K", ""))

In [136]:
players[['name', 'market_value']].head(3)

,name,market_value
0,Luciano Gondou,7.4000
1,Maximiliano Romero,2.5000
2,Gastón Verón,1.2000


#### 2. Xử lý dữ liệu 'age'

In [137]:
players[['name', 'age']].head(3)

,name,age
0,Luciano Gondou,22 yrs
1,Maximiliano Romero,25 yrs
2,Gastón Verón,22 yrs


In [138]:
# Sửa cột dữ liệu 'age'
players['age'] = players['age'].apply(lambda x: x.split(" ")[0])

In [139]:
players[['name', 'age']].head(3)

,name,age
0,Luciano Gondou,22
1,Maximiliano Romero,25
2,Gastón Verón,22


#### 3. Xử lý dữ liệu 'height'

In [140]:
players[['name', 'height']].head(3)

,name,height
0,Luciano Gondou,189 cm
1,Maximiliano Romero,179 cm
2,Gastón Verón,184 cm


In [141]:
players['height'] = players['height'].apply(lambda x: x.split(" ")[0])

In [142]:
players[['name', 'height']].head(3)

,name,height
0,Luciano Gondou,189
1,Maximiliano Romero,179
2,Gastón Verón,184


### ** Xóa bỏ cột dư thừa **

#### 1. Xóa bỏ cột dư thừa của bảng 'countries'

In [143]:
countries.head(2)

,rank,name,name_code,points,slug,category_id,tournament_name,tournament_slug,tournament_id,tournament_season_id
0,1,Argentina,ARG,1858.00,argentina,48.0,Liga Profesional de Fútbol,liga-profesional-de-futbol,155.0,57478.0
1,2,France,FRA,1840.59,france,7.0,Ligue 1,ligue-1,34.0,52571.0


In [144]:
countries = countries.drop(['slug', 'category_id', 'tournament_slug', 'tournament_id', 'tournament_season_id'], axis=1)

In [145]:
countries.head(2)

,rank,name,name_code,points,tournament_name
0,1,Argentina,ARG,1858.00,Liga Profesional de Fútbol
1,2,France,FRA,1840.59,Ligue 1


#### 2. Xóa bỏ các cột dư thừa của bảng 'clubs'

In [146]:
clubs.head(2)

,country,tournament,name,short_name,name_code,position,slug,id
0,Argentina,Liga Profesional de Fútbol,Argentinos Juniors,Argentinos Jrs.,ARG,1,argentinos-juniors,3216
1,Argentina,Liga Profesional de Fútbol,Atlético Tucumán,Atlético Tucumán,CAT,2,atletico-tucuman,36833


In [147]:
clubs = clubs.drop(['slug', 'id'], axis=1)
clubs = clubs[['name', 'short_name', 'name_code', 'position', 'tournament', 'country']]

In [148]:
clubs.head(2)

,name,short_name,name_code,position,tournament,country
0,Argentinos Juniors,Argentinos Jrs.,ARG,1,Liga Profesional de Fútbol,Argentina
1,Atlético Tucumán,Atlético Tucumán,CAT,2,Liga Profesional de Fútbol,Argentina


#### 3. Xóa bỏ cột dư thừa của bảng 'players'

In [149]:
players.columns

Index(['name', 'market_value', 'club', 'country', 'league', 'nationality',
       'birth_date', 'age', 'height', 'preferred_foot', 'position',
       'shirt_number', 'attacking', 'technical', 'tactical', 'defending',
       'creativity', 'saves', 'anticipation', 'ball_distribution', 'aerial',
       'avg_score_last_12_months', 'appearances_last_12_months'],
      dtype='object')

In [150]:
players = players.drop(['birth_date'], axis=1)

In [151]:
players.columns

Index(['name', 'market_value', 'club', 'country', 'league', 'nationality',
       'age', 'height', 'preferred_foot', 'position', 'shirt_number',
       'attacking', 'technical', 'tactical', 'defending', 'creativity',
       'saves', 'anticipation', 'ball_distribution', 'aerial',
       'avg_score_last_12_months', 'appearances_last_12_months'],
      dtype='object')

### ** Chuyển đổi kiểu dữ liệu  **

In [152]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11814 entries, 0 to 12420
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        11814 non-null  object 
 1   market_value                11814 non-null  object 
 2   club                        11814 non-null  object 
 3   country                     11814 non-null  object 
 4   league                      11814 non-null  object 
 5   nationality                 11814 non-null  object 
 6   age                         11814 non-null  object 
 7   height                      11814 non-null  object 
 8   preferred_foot              11814 non-null  object 
 9   position                    11814 non-null  object 
 10  shirt_number                11814 non-null  int64  
 11  attacking                   11814 non-null  int64  
 12  technical                   11814 non-null  int64  
 13  tactical                    11814 no

In [153]:
players[['market_value', 'age', 'height']] = players[['market_value', 'age', 'height']].apply(pd.to_numeric, errors='coerce')

In [154]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11814 entries, 0 to 12420
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        11814 non-null  object 
 1   market_value                11814 non-null  float64
 2   club                        11814 non-null  object 
 3   country                     11814 non-null  object 
 4   league                      11814 non-null  object 
 5   nationality                 11814 non-null  object 
 6   age                         11814 non-null  int64  
 7   height                      11814 non-null  int64  
 8   preferred_foot              11814 non-null  object 
 9   position                    11814 non-null  object 
 10  shirt_number                11814 non-null  int64  
 11  attacking                   11814 non-null  int64  
 12  technical                   11814 non-null  int64  
 13  tactical                    11814 no

### ** Ghi lại file data clean **

In [157]:
players.to_csv('./clean_data/players.csv', index=False)
clubs.to_csv('./clean_data/clubs.csv', index=False)
countries.to_csv('./clean_data/countries.csv', index=False)